In [ ]:
import numpy as np

from numpy import random

from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup
from bokeh.models import CustomJS, HoverTool
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.driving import count

from bokeh.models import Range1d

import time
import serial

# create a data source
source = ColumnDataSource(dict(
    timeList=[],
    singleCounts=[]
    ))

# make the plot and its attributes

singleCountsPlot = figure(plot_height=260, plot_width=1000, title="Single Counts",
              tools="crosshair,pan,reset,save,wheel_zoom")

singleCountsPlot.x_range.follow = "end"
singleCountsPlot.x_range.follow_interval = 20
singleCountsPlot.x_range.range_padding = 0.1
singleCountsPlot.x_range.range_padding_units = 'absolute'

singleCountsCircles = singleCountsPlot.circle(x = 'timeList', y = 'singleCounts', source = source, 
                                              line_width=6, line_alpha=0.6, line_color = 'orange')

singleCountsPlot.line(x = 'timeList', y = 'singleCounts', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'orange')

singleCountsPlot.add_tools(HoverTool(renderers=[singleCountsCircles], tooltips=[("Time","@timeList"), 
                        ("Counts","@singleCounts")], mode='mouse'))

singleCountsPlot.y_range = Range1d(-1,101)

# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1, width = 200)

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0

# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    global runningTime
    
    # generate a random integer between 0 and 100
    randomValue = np.random.randint(0,100)
            
    
    # update the data source
    new_data = dict(
        timeList=[runningTime],
        singleCounts=[randomValue]
        )
    
    # Stream the new data to the plots.
    source.stream(new_data)

    # if the random number is less than 31, then draw a gray point over the other point.
    if randomValue < 31:
        
        falseCircle = singleCountsPlot.circle(x = runningTime, y = randomValue, 
                            line_width=6, line_alpha=1.0, line_color = 'darkgrey')
    
        singleCountsPlot.add_tools(HoverTool(renderers=[falseCircle], tooltips=[("Time", str(runningTime)), 
                        ("Error","False Zero")], mode='mouse'))
    
    runningTime = runningTime + 1
    
    
# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        callback_id = curdoc().add_periodic_callback(update_data, 1000)
    
    # else if the STOP button is on
    elif startStopButtonGroup.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
# When the START/STOP button is clicked, call the change_periodic_callback function
startStopButtonGroup.on_click(change_periodic_callback)


pageLayout = row([startStopButtonGroup, singleCountsPlot])

curdoc().add_root(pageLayout)

curdoc().title = "Color Change"